In [1]:
import pandas as pd
from modeling import feature_engineering, get_best_params,closest, agg_function,agg_diff_fun, save_object, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
metro= pd.read_csv('MetroStanice.csv')
df_train["training"]=1
df_test["training"]=0

df= pd.concat([df_train,df_test],sort=True).reset_index(drop=True)

mapping_features= pd.read_csv('feature_deduplication.csv')
mydict= dict(zip(mapping_features.original_feature, mapping_features.unique_feature))
#features and photos
df["len_features"] = df["features"].apply(len)
df["len_photos"] = df["photos"].apply(len)
df['features']= df['features'].apply(lambda x: [item.lower() for item in x]) 
df['features_txt']= df['features'].apply(lambda x: ' '.join(x))
df['features']=df['features'].apply(lambda x: [mydict[item] if item in mydict else item for item in x ])
#pocet features
#

#timefromepoch
import datetime
#df['tstamp'] =df['created'].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
#Datumove
#df["date"]=  df['created'].str[:7]
df['created']=pd.to_datetime(df["created"])
#df['day'] = df['created'].dt.day
#df['days']=(df["created"]-min(df["created"])).apply(lambda x: x.days)
df['dayofweek'] = df['created'].dt.dayofweek
#cylicke
df['hr'] = df['created'].dt.hour
df['mnth'] = df['created'].dt.month
df['nwords_description']= df['description'].apply(lambda x: x.count(' ')+1)
df['price_digits']=df['price'].apply(lambda x: len(str(x)))
df['price'] = np.log(df['price'])

# GEO
coordinates= df[["latitude","longitude"]].values
coordinates_tuple=tuple((x[0],x[1]) for x in coordinates)
adresy=revgc.search(coordinates_tuple)
df=pd.concat([df,pd.DataFrame(adresy)],axis=1)


#label encoding
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df['interest_integer'] =df['interest_level'].map(event_dictionary) 

#interakcie zakladna intuicia
df['bed+bath']=df['bedrooms']+df['bathrooms']
df['price/bed']=df['price']/(df['bedrooms']+1e-16)
df['price/bed+bath']=df['price']/(df['bed+bath']+1e-16)

training='training'
label='interest_integer'

metro=metro.loc[:,['lat','lon']]
metro=metro.to_dict('records')
dfmetro=df.loc[:,['latitude','longitude']]
dfmetro=dfmetro.to_dict('records')
x=[]
for i in dfmetro:
    x.append(closest(metro, i))
df['metro']=x


#drop columns
df=df.drop(columns=['photos','created','listing_id','lat','lon','latitude','longitude'])

Loading formatted geocoded file...


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
regression=LogisticRegression(random_state=0,solver='lbfgs',multi_class='auto',n_jobs=-1,fit_intercept=True,max_iter=100)
steps = [('scaler', StandardScaler()), ('Log', regression)]
pipeline = Pipeline(steps)
clf=pipeline
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis')

2020-05-22 22:53:44.213887


100%|██████████| 13/13 [00:30<00:00,  2.32s/it]


after numeric feature selection:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'price_digits', 'bed+bath', 'price/bed', 'price/bed+bath'] 0.6935492633825219


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N features:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'price_digits', 'bed+bath', 'price/bed', 'price/bed+bath', 'laundry', 'hardwood', 'cats allowed', 'dogs allowed', 'dish washer', 'no fee', 'fitness center', 'pre war', 'outdoor space', 'high ceiling', 'dining room', 'balcony', 'exclusive', 'loft', 'common outdoor space', 'simplex', 'low-rise', 'garage', 'reduced fee', 'furnished', 'multi level', 'private outdoor space', 'park view', 'washer & dryer', 'renovated', 'pool', 'on-site laundry', 'stainless steel', 'dryer in unit', 'valet', 'residents garden', 'on-site garage', 'walk in closet', 'light', 'patio'] 0.6602109961742875


...

3 <function stemming_tokenizer at 0x7f55792713b0> 1
3 <function stemming_tokenizer at 0x7f55792713b0> 2
0.5651622632201556
3 <function stemming_tokenizer at 0x7f55792713b0> 3
3 <function stemming_tokenizer at 0x7f55792713b0> 4
3 <function stemming_tokenizer at 0x7f55792713b0> 5


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


3 <modeling.LemmaTokenizer object at 0x7f556fdb12d0> 1
3 <modeling.LemmaTokenizer object at 0x7f556fdb12d0> 2
3 <modeling.LemmaTokenizer object at 0x7f556fdb12d0> 3
3 <modeling.LemmaTokenizer object at 0x7f556fdb12d0> 4
3 <modeling.LemmaTokenizer object at 0x7f556fdb12d0> 5
3 None 1
3 None 2
3 None 3
3 None 4


100%|██████████| 3/3 [1:04:01<00:00, 1280.66s/it]

3 None 5
after text feature tokenization plus PCA:  ['bathrooms', 'bedrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'price_digits', 'bed+bath', 'price/bed', 'price/bed+bath', 'laundry', 'hardwood', 'cats allowed', 'dogs allowed', 'dish washer', 'no fee', 'fitness center', 'pre war', 'outdoor space', 'high ceiling', 'dining room', 'balcony', 'exclusive', 'loft', 'common outdoor space', 'simplex', 'low-rise', 'garage', 'reduced fee', 'furnished', 'multi level', 'private outdoor space', 'park view', 'washer & dryer', 'renovated', 'pool', 'on-site laundry', 'stainless steel', 'dryer in unit', 'valet', 'residents garden', 'on-site garage', 'walk in closet', 'light', 'patio', 'mean_bathrooms_by_building_id', 'median_bathrooms_by_building_id', 'sum_bathrooms_by_building_id', 'std_bathrooms_by_building_id', 'bathrooms-mean_bathrooms_by_building_id', 'bathrooms-median_bathrooms_by_building_id', 'mean_price_by_building_id', 'median_price_by_build

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
regression=LogisticRegression(random_state=0,solver='lbfgs',multi_class='auto',n_jobs=-1,fit_intercept=True,**best_params)
steps = [('scaler', StandardScaler()), ('Log', regression)]
pipeline = Pipeline(steps)
clf=pipeline
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=100
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)


2020-04-29 10:11:38.371748


100%|██████████| 12/12 [00:22<00:00,  1.90s/it]


after numeric features selection:  ['bedrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'price_digits', 'bed+bath', 'price/bed', 'price/bed+bath'] 0.6935139676261675


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N feature:  ['bedrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'price_digits', 'bed+bath', 'price/bed', 'price/bed+bath', 'elevator', 'cats allowed', 'hardwood floors', 'dogs allowed', 'dishwasher', 'laundry in building', 'no fee', 'fitness center', 'laundry in unit', 'pre-war', 'outdoor space', 'dining room', 'high speed internet', 'balcony', 'new construction', 'terrace', 'exclusive', 'loft', 'prewar', 'common outdoor space', 'hardwood', 'simplex', 'lowrise', 'garage', 'reduced fee', 'laundry room', 'furnished', 'multi-level', 'high ceilings', 'private outdoor space', 'parking space', 'roof-deck', 'renovated', 'pool', 'on-site laundry', 'high ceiling', 'dryer in unit', 'concierge', 'on-site garage', 'patio', 'washer/dryer', 'granite kitchen', 'walk in closet(s)', 'subway', 'live-in super', 'common parking/garage', 'wifi access', 'parking', 'highrise', 'lounge', 'short term allowed', 'childrens playroom', 'no pets', 'actual a

...

building_id count_encoder 0.5921008469997849
building_id label_encoder 0.5920971140815435
building_id CB_target_encoding 0.5848796160687426


 14%|█▍        | 1/7 [10:07<1:00:46, 607.82s/it]

manager_id count_encoder 0.584835780414196
manager_id CB_target_encoding 0.5631647818739416


 43%|████▎     | 3/7 [20:38<34:40, 520.01s/it]  

manager_id LOO_target_encoding 0.5622693672277851
name count_encoder 0.562260466788643
name label_encoder 0.5621883947470679


In [ ]:
parameters=pd.read_csv('Pipeline_grid_results.csv')
best_params=get_best_params(parameters,'score')
best_params
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_best_grid_params')